In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical
from copy import deepcopy

In [2]:
import import_ipynb
import pandas as pd
import numpy as np

In [3]:
import Model_DomainIL
from Model_DomainIL import *

In [4]:
import utils
from utils import *

In [5]:
import ewc_DomainIL
from ewc_DomainIL import *

In [6]:
def train_loop_domain():
    return 0

In [7]:

def train_loop_domain(model, OPTIMIZER, MAX_LABEL, cw_data, ow_data, test_size,
                first_task = 49, inc_task = 45, first_epochs = 30, inc_epochs = 5,
                  lamb=100, num_sample=100):
    
    first_part = split_by_label(cw_data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) <= MAX_LABEL:
            
            if i == 0:
                model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = np.ones_like(train_label) # 모두 1로 수정
                

                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=1)
                print(f"   First_task training accuracy: {history.history['accuracy'][-1]:.4f}")

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(cw_data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)

                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb), optimizer=OPTIMIZER, metrics=["accuracy"])
                
                # 3D ndarray 로 변환 (이부분 함수로 바꾸기)
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = np.ones_like(train_label) # 모두 1로 수정


                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=1)
                print(f"   {i}_task training accuracy: {history.history['accuracy'][-1]:.4f}")


                test_combined = pd.concat([test, ow_data])
                test_combined = test_combined.sample(frac=1, random_state=42).reset_index(drop=True)

                test_seq, test_label = split_data_label(test_combined)
                test_seq = np.stack(test_seq)
                test_seq = test_seq[..., np.newaxis]

                test_label = test_label.values
                test_label = np.ones_like(test_label) # 모두 1로 수정

                test_ = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
                test_ = test_.batch(32) #(수정) 모델 자체 배치 존재? - 학습시 fit 디폴트값도 32

                inc_accuracy = evaluate(model, test_)
                print(f"Task {i} accuracy after training on Task ~{i-1}: {inc_accuracy:.4f}")




                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

        else:
            break 